In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

import os, sys

#from arrow import get
sys.path.append('/home/hrai/codes/hpe_library')
from lib_import import *
from my_utils import BatchCamera, Camera
from my_utils import arg_as_list, get_configs
from my_utils import generate_random_segment, generate_random_trajectory, get_aligned_init_torso, get_ap_pose_2d, get_backbone_line_from_torso, get_bounded_segments_idx, get_cam_param, get_input, get_label, get_model_input, get_output, get_pairs, get_part_traj, get_pose_seq_and_cam_param, get_two_point_parts, load_h36m, load_segment_file_from_parameters, make_input, make_one_dimension_list, make_output, split_continuous_indices, MyCustomDataset, parse_args_by_model_name, get_input_gt_for_onevec, get_limb_angle, get_h36m_camera_info, h36m_data_processing
from my_utils import DH_matrix, azim_elev_to_vec, batch_azim_elev_to_vec, batch_inverse_tf, batch_projection, batch_rot_x_matrix, batch_rot_y_matrix, batch_rot_z_matrix, calculate_azimuth_elevation, calculate_rotation_quaternion, draw_arm, draw_subline, draw_torso_direction, frame_vec_to_matrix, generate_batch_tf_from_batch_origin_R, generate_camera_frame, generate_dh_frame, generate_tf_from_origin_R, generate_two_link, generate_world_frame, get_batch_frame_vec_from_keypoints, get_batch_lower_torso_frame_from_keypoints, get_batch_lower_torso_frame_from_pose, get_batch_reference_frame, get_batch_upper_torso_frame_from_keypoints, get_batch_upper_torso_frame_from_pose, get_frame_from_keypoints, get_lower_torso_frame_from_pose, get_reference_frame, get_torso_direction, get_torso_rotation_matrix, get_torso_shape, get_upper_torso_frame_from_pose, inverse_tf, normalize_vector, project_batch_tensor, projection, rotate_torso_by_R, rotate_torso_by_R_for_batch_tensor, rotation_matrix_to_vector_align, rotation_matrix_torso2torso, Appendage, BatchAppendage, BatchDHModel, DHModel, calculate_batch_azimuth_elevation
from my_utils import args_dict_to_namespace, construct_torso_from_output, denormalize_motionbert_result, get_dataset_info_from_segment_folder, get_inference_from_dhdst, get_inference_from_dhdst_torso, get_inference_from_motionbert, get_output_type, get_result, infer_one_segment, load_best_model_for_inference, normalize_input, test_model_by_segment_file
from my_utils import get_logger, log_configs
from my_utils import init_weights, split_array_by_idxs, BaselineModel, Linear, TorsoModel
from my_utils import Camera2ImageCoordinate, MPJPE, MPJPE_for_multiple_pose, MPJPE_for_single_pose, World2CameraCoordinate, World2ImageCoordinate, _sqrt_positive_part, _weak_project, aihub2h36m, array2dict, avgErrorForOneAction, avgErrorForOneActor, avgErrorForOneCamera, camera_to_image_frame, change_bbox_convention, check_max_min, coco2h36m, convert3DResult, draw_skeleton, draw_skeleton_2d, draw_skeleton_both, euclidean_distance, fit3d2h36m, fit3d_load_gt_and_param, get3DResult, getAIHubCameraParameter, getGT, getNumFromImgName, get_batch_h36m_keypoints, get_bbox_area, get_bbox_area_from_pose2d, get_bbox_from_pose2d, get_h36m_keypoint_index, get_h36m_keypoints, get_rootrel_pose, get_video_frame, get_video_info, get_xy_centered_pose, halpe2h36m, infer_box, kookmin2h36m, kookmin2h36m_with_nose, loadAIHubCameraParameter, matrix_to_quaternion, normalize, normalize_array, optimize_scaling_factor, plot_cv2_image, readJSON, readpkl, savepkl, skew_symmetric_matrix, skew_symmetric_matrix_tensor, standardize_quaternion, mpi_inf_3dhp2h36m, get_pose_height, get_batch_bbox_from_pose2d, len_ids, part_ids, h36m_part_keypoints, get_h36m_limb_lens, procrustes_align
from my_utils import check_duplicate_training, get_input_output_candidate, get_num_trial, load_args, load_best_model, load_dataset, load_model, prepare_training, run, run_epoch, save_args, split_array_by_idxs
from my_utils import axes_2d, axes_3d, axes_to_compare_pred_gt, draw_3d_pose, draw_bbox, draw_multiple_2d_pose, draw_multiple_3d_pose, draw_one_segment, draw_rotation_matrix, draw_segment, draw_segments, draw_trajectory, get_2d_pose_image, legend_without_duplicate_labels, plot_to_compare_pred_gt, save_h36m_pose_video, show2Dtrajectory, show3Dtrajectory, show_2d_3d, show_whole_segment_trajectories, draw_2d_pose, clear_axes
from my_utils import check_available_frame, check_continuity, draw_base_marker_3d, generate_kookmin_pkl_for_each_video, get_cam_param_kookmin, get_video_frame_kookmin, get_video_num_frame_kookmin, load_pose3d_kookmin, load_csv_kookmin, get_lbot
os.chdir('/home/hrai/codes/MotionBERT')

from lib.utils.tools import * # get_config
from lib.utils.learning import * # load_backbone
from lib.utils.args import parse_args, check_args
from lib.utils.utils_data import flip_data
from lib.model.loss import *
from lib.model.evaluation import *
from lib.data.datareader_kookmin import DataReaderKOOKMIN
from lib.data.datareader_h36m import DataReaderH36M
from lib.data.datareader_fit3d import DataReaderFIT3D
from lib.data.datareader_random_limb import DataReaderRandomLimb
from lib.data.dataset_motion_3d import MotionDataset3D
from lib.model.DHDSTformer import DHDSTformer_total, DHDSTformer_total2, DHDSTformer_total3, DHDSTformer_limb, DHDSTformer_torso, DHDSTformer_torso2, DHDSTformer_total4

import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, interactive_output
import threading
from ipywidgets import GridspecLayout
from ipywidgets import TwoByTwoLayout

from utils_result_analysis.button import get_inference_button, get_visualize_button, get_root_rel_button, get_reset_button, get_go_to_max_frame_button, get_procrustes_button, get_analysis_error_button, get_analysis_dh_button
from utils_result_analysis.progress import get_inference_progress
from utils_result_analysis.text import get_inputs_all_shape_text, get_gts_all_shape_text, get_results_all_shape_text, get_error_max_frame_text, get_batch_num_text, get_frame_num_text
from utils_result_analysis.slider import get_azim_slider, get_elev_slider, get_zoom_slider, get_delay_slider, get_frame_slider
from utils_result_analysis.dropdown import get_model_list_dropdown, get_dataset_list_dropdown
from utils_result_analysis.play import get_play_vis_button
from utils_result_analysis.select import get_subject_select, get_action_select, get_cam_select, get_batch_select, get_part_select

box_layout = widgets.Layout(
        border='solid 1px red',
        margin='0px 10px 10px 0px',
        padding='5px 5px 5px 5px')
 
plt.clf()
plt.cla()

class analysis_tool():
    def __init__(self):
        # variables -------------------------------------------------------------------
        self.stop_inference_flag = False
        self.inference_complete = False
        self.joint_names = ['pelvis', 'right_hip', 'right_knee', 'right_ankle', 'left_hip', 'left_knee', 'left_ankle', 'torso', 'neck', 'nose', 'head', 'left_shoulder', 'left_elbow', 'left_wrist', 'right_shoulder', 'right_elbow', 'right_wrist']
        self.part_list = ['R_UPPER_ARM', 'R_UNDER_ARM', 'L_UPPER_ARM', 'L_UNDER_ARM', 'R_UPPER_LEG', 'R_UNDER_LEG', 'L_UPPER_LEG', 'L_UNDER_LEG']
        self.data_dict = {}
        self.batch_input = None
        self.action_clips = None
        self.prev_batch_analysis_error = -1
        self.prev_frame_analysis_error = -1
        self.prev_batch_analysis_dh = -1
        self.prev_frame_analysis_dh = -1
        self.prev_part_ids = ['']
        self.test_out = widgets.Output()
        try:
            # model, data preparation -----------------------------------------------------
            self.load_panel()
            # select ---------------------------------------------------------------------
            self.select_panel()
            # inference -------------------------------------------------------------------
            self.inference_panel()
            # vis_control -----------------------------------------------------------------
            self.vis_control_panel()
            # figure ----------------------------------------------------------------------
            self.figure_panel()
            # analysis --------------------------------------------------------------------
            self.analysis_panel()
            # interactive -----------------------------------------------------------------
            self.interactive()
            # layout ----------------------------------------------------------------------
            self.layout()
            # main
            self.main()
        except Exception as e:
            with self.test_out:
                print(e)
            
    # panel functions -------------------------------------------------------------------  
    def load_panel(self):
        self.model_list = get_model_list_dropdown(['MB_release', 'MB_ft_h36m', 'MB_train_h36m_with_canonical2', 'MB_ft_h36m_dh_angle1', 'MB_ft_h36m_dh_angle1_with_canonical', 'MB_train_h36m_gt_world_no_factor', 'MB_train_h36m_gt_cam_no_factor', 'DHDSTformer_total4', 'DHDST2_ft_h36m_gt_cam_no_factor_baseline'])
        self.dataset_list = get_dataset_list_dropdown(['H36M-SH', 'H36M-GT', 'FIT3D', 'KOOKMIN', 'H36M-GT-WORLD_NO_FACTOR', 'H36M-GT-CAM_NO_FACTOR'])
        
    def inference_panel(self):
        # inference button
        self.button_inference = get_inference_button()
        self.button_inference.on_click(self.inference)
        # inference progress
        self.result_out = widgets.Output()      
        self.inference_progress = get_inference_progress()
        # result shapes
        self.inputs_all_shape = get_inputs_all_shape_text()
        self.gts_all_shape = get_gts_all_shape_text()
        self.results_all_shape = get_results_all_shape_text()
        self.inputs_all_shape.layout.width = 'max-content'
        self.gts_all_shape.layout.width = 'max-content'
        self.results_all_shape.layout.width = 'max-content'

    def select_panel(self):
        self.select_subject = get_subject_select([''])
        self.select_action = get_action_select([''])
        self.select_cam = get_cam_select([''])
        self.select_batch = get_batch_select([''])
        self.select_subject.layout.width = 'max-content'
        self.select_action.layout.width = 'max-content'
        self.select_cam.layout.width = 'max-content'
        self.select_batch.layout.width = 'max-content'
        
    def vis_control_panel(self):
        # batch/frame selection
        #self.batch_num = get_batch_num_text()
        self.frame_num = get_frame_num_text()
        # visualize button
        #self.button_visualize = get_visualize_button()
        # root rel button
        self.button_root_rel = get_root_rel_button()
        # procrustes button
        self.button_procrustes = get_procrustes_button()
        # analysis_error button
        self.button_analysis_error = get_analysis_error_button()
        # analysis dh button
        self.button_analysis_dh = get_analysis_dh_button()
        # view control
        self.azim = get_azim_slider()
        self.elev = get_elev_slider()
        self.zoom = get_zoom_slider()
        # play control
        self.delay = get_delay_slider()
        self.play_vis = get_play_vis_button()
        self.error_max_frame = get_error_max_frame_text() # max error frame
        self.button_go_to_max_frame = get_go_to_max_frame_button() # go to max frame button
        # reset buttons
        self.button_frame_reset = get_reset_button('Reset frame')
        self.button_view_reset = get_reset_button('Reset view')
        self.button_delay_reset = get_reset_button('Reset delay')
        self.button_figure_reset = get_reset_button('Reset figure')
        
        # link widgets
        widgets.jslink((self.play_vis, 'value'), (self.frame_num, 'value'))
        widgets.jslink((self.play_vis, 'interval'), (self.delay, 'value')) # change interval when fps changes
        self.button_frame_reset.on_click(self.frame_reset)
        self.button_view_reset.on_click(self.view_reset)
        self.button_delay_reset.on_click(self.delay_reset)
        self.button_figure_reset.on_click(self.clear_figure)
        #self.analysis_interact = interactive(self.analysis_error, batch_num=self.batch_num, frame_num=self.frame_num, root_rel=self.button_root_rel, analysis=self.button_analysis)
        self.button_go_to_max_frame.on_click(self.go_to_max_frame)
        
    def figure_panel(self):
        # figure
        self.plot_input = widgets.Output(layout=box_layout)
        self.plot_3d = widgets.Output(layout=box_layout)
        self.vis_plot_init()

    def analysis_panel(self):
        # frame slider
        self.frame_slider = get_frame_slider(243)
        # analysis figure
        self.analysis_error = widgets.Output(layout=box_layout, width='auto')
        self.analysis_dh = widgets.Output(layout=box_layout, width='auto')
        self.analysis_plot_init()
        # part select
        self.select_part = get_part_select(self.part_list)
        self.select_part.layout.width = 'max-content'
        # link widgets
        widgets.jslink((self.frame_slider, 'value'), (self.frame_num, 'value'))

    def vis_plot_init(self):
        with self.plot_input:
            self.fig_input = plt.figure(0, figsize=(3, 3), layout='tight')
            self.fig_input.clear()
            self.ax_input = axes_2d(self.fig_input, normalize=True)
            f = plt.show()
        with self.plot_3d:
            self.fig_3d = plt.figure(1, figsize=(5, 15),  layout='tight')
            self.fig_3d.clear()
            self.ax_3d1 = axes_3d(self.fig_3d, loc=211, xlim=(-1,1), ylim=(-1,1), zlim=(-1,1), view=(0,0), show_axis=False)      
            self.ax_3d2 = axes_3d(self.fig_3d, loc=212, xlim=(-1,1), ylim=(-1,1), zlim=(-1,1), view=(0, 90), show_axis=False)
            self.ax_3d1.set_aspect('equal')    
            self.ax_3d2.set_aspect('equal')  
            f = plt.show()


    def analysis_plot_init(self):
        with self.analysis_error:
            self.fig2 = plt.figure(3, figsize=(7, 7), layout='tight')
            self.fig2.clear()
            self.ax_error_per_frame = self.fig2.add_subplot(211)
            self.ax_error_per_joint = self.fig2.add_subplot(212)
            self.ax_error_per_frame.set_title('Joint error per frame')
            self.ax_error_per_frame.set_xlabel('Frame')
            self.ax_error_per_joint.set_title('Joint error per joint')
            self.ax_error_per_joint.set_xlabel('Joint')
            f = plt.show()
        with self.analysis_dh:
            self.fig3 = plt.figure(4, figsize=(10, 10), layout='tight')
            self.fig3.clear()
            self.ax_length = self.fig3.add_subplot(311)
            self.ax_azim = self.fig3.add_subplot(312)
            self.ax_elev = self.fig3.add_subplot(313)
            self.ax_azim.set_ylim(0, np.pi)
            self.ax_elev.set_ylim(0, np.pi)
            f = plt.show()

            
    # button functions -------------------------------------------------------------------
    def view_reset(self, b):
        self.azim.value = -90
        self.elev.value = 0
        self.zoom.value = 5.0
        #self.vis_interact.update()    
    
    def frame_reset(self, b):
        self.frame_num.value = 1
        #self.vis_interact.update()

    def delay_reset(self, b):
        self.delay.value = 33.33 # 30fps
        #self.vis_interact.update()
        
    def go_to_max_frame(self, b):
        self.frame_num.value = self.error_max_frame.value
        
    def load_model(self, target_model):
        if target_model == 'DHDSTformer_total4': target_model = 'DHDST4_h36m_gt_cam_no_factor_baseline'
        opts = parse_args_by_model_name(target=target_model)
        args = get_config(opts.config)
        args = check_args(args, opts)
        print(f'model: {args.model} \n loc: {opts.evaluate}')
        
        if 'DHDSTformer_total2' == args.model: model_pos = DHDSTformer_total2(chk_filename='', args=args)
        elif 'DHDSTformer_torso' == args.model: model_pos = DHDSTformer_torso(chk_filename='', args=args)
        elif 'DHDSTformer_torso2' == args.model: model_pos = DHDSTformer_torso2(chk_filename='', args=args)
        elif 'DHDSTformer_limb' == args.model: model_pos = DHDSTformer_limb(chk_filename='', args=args)
        elif 'DHDSTformer_total4' == args.model: model_pos = DHDSTformer_total4(args=args)
        elif 'MB' in opts.evaluate: model_pos = load_backbone(args)
        
        model_pos = nn.DataParallel(model_pos)
        if torch.cuda.is_available():
            model_pos = model_pos.cuda()
        checkpoint = torch.load(opts.evaluate, map_location=lambda storage, loc: storage)
        model_pos.load_state_dict(checkpoint['model_pos'], strict=True)
        print('Model Loaded!')
        self.model_pos = model_pos
        self.args = args
        
    def load_data(self, target_dataset):
        args = self.args
        self.target_dataset = target_dataset
        if target_dataset == 'H36M-SH':
            args.dt_file = 'h36m_sh_conf_cam_source_final.pkl'
            args.subset_list = ['H36M-SH']
            self.datareader = DataReaderH36M(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file)
            testloader_params = {
                'batch_size': args.batch_size,
                'shuffle': False,
                'num_workers': 12,
                'pin_memory': True,
                'prefetch_factor': 4,
                'persistent_workers': True
            }
            self.test_dataset = MotionDataset3D(args, args.subset_list, 'test')
            self.test_loader = DataLoader(self.test_dataset, **testloader_params)
            print(f'{target_dataset} Loaded!')
        elif target_dataset == 'H36M-GT-WORLD_NO_FACTOR':
            args.dt_file = 'h36m_gt_no_factor.pkl'
            args.subset_list = ['H36M-GT_WORLD_NO_FACTOR']
            self.datareader = DataReaderH36M(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file, mode='world_3d')
            testloader_params = {
                'batch_size': args.batch_size,
                'shuffle': False,
                'num_workers': 12,
                'pin_memory': True,
                'prefetch_factor': 4,
                'persistent_workers': True
            }
            self.test_dataset = MotionDataset3D(args, args.subset_list, 'test')
            self.test_loader = DataLoader(self.test_dataset, **testloader_params)
            print(f'{target_dataset} Loaded!')
        elif target_dataset == 'H36M-GT-CAM_NO_FACTOR':
            args.dt_file = 'h36m_gt_no_factor.pkl'
            args.subset_list = ['H36M-GT-CAM_NO_FACTOR']
            self.datareader = DataReaderH36M(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file, mode='cam_3d')
            testloader_params = {
                'batch_size': args.batch_size,
                'shuffle': False,
                'num_workers': 12,
                'pin_memory': True,
                'prefetch_factor': 4,
                'persistent_workers': True
            }
            self.test_dataset = MotionDataset3D(args, args.subset_list, 'test')
            self.test_loader = DataLoader(self.test_dataset, **testloader_params)
            print(f'{target_dataset} Loaded!') 
        elif target_dataset == 'FIT3D':
            args.dt_file = 'fit3d_all_train_except_target_action.pkl'
            args.subset_list = ['FIT3D_ALL_TRAIN_EXCEPT_TARGET_ACTION']
            self.datareader = DataReaderFIT3D(n_frames=args.clip_len, sample_stride=args.sample_stride, data_stride_train=args.data_stride, data_stride_test=args.clip_len, dt_root = 'data/motion3d', dt_file=args.dt_file, mode='cam_3d')
            testloader_params = {
                'batch_size': args.batch_size,
                'shuffle': False,
                'num_workers': 12,
                'pin_memory': True,
                'prefetch_factor': 4,
                'persistent_workers': True
            }
            self.test_dataset = MotionDataset3D(args, args.subset_list, 'test')
            self.test_loader = DataLoader(self.test_dataset, **testloader_params)
            print(f'{target_dataset} Loaded!') 
        elif target_dataset == 'KOOKMIN':
            print('KOOKMIN is not implemented yet')
        
            
    def inference(self, b):
        self.result_out.clear_output()
        args = self.args
        model_pos = self.model_pos
        test_loader = self.test_loader

        model_pos.eval() 
        if not os.path.exists('/home/hrai/codes/MotionBERT/saved_results/'):
            os.makedirs('/home/hrai/codes/MotionBERT/saved_results/')
        save_path = f'/home/hrai/codes/MotionBERT/saved_results/result_dict_{args.model}_{self.target_dataset}.pkl'
        if os.path.exists(save_path):
            with self.result_out:
                print('Loading saved results...')
            result_dicts = readpkl(save_path)
            self.results_all = result_dicts['results_all']
            self.gts_all = result_dicts['gts_all']
            self.inputs_all = result_dicts['inputs_all']
        else:
            results_all = []
            gts_all = []
            inputs_all = []
            num_total_batch = len(test_loader)
            batch_step = 0
            with torch.no_grad():
                for batch_input, batch_gt in test_loader: # batch_input: normalized joint_2d, batch_gt: normalized joint3d_image
                    batch_size = len(batch_input) 
                    # preprocessing
                    batch_input, batch_gt, batch_gt_torso, batch_gt_limb_pos = preprocess_eval(args, batch_input, batch_gt)
                    # inference
                    predicted_3d_pos = batch_inference_eval(args, model_pos, batch_input, batch_gt_torso, batch_gt_limb_pos)
                    # postprocessing
                    if args.rootrel:
                        predicted_3d_pos[:,:,0,:] = 0     # [N,T,17,3]
                    else:
                        batch_gt[:,0,0,2] = 0
                    if args.gt_2d: # input 2d를 추론값으로 사용함으로써 depth만 추정하도록 함
                        predicted_3d_pos[...,:2] = batch_input[...,:2]
                    # store the results
                    results_all.append(predicted_3d_pos.cpu().numpy())
                    gts_all.append(batch_gt.cpu().numpy())
                    inputs_all.append(batch_input.cpu().numpy())
                        
                    batch_step += 1
                    batch_percentage = (batch_step/num_total_batch)*100
                    self.inference_progress.value = batch_percentage
                    self.inference_progress.description = f'{batch_percentage:.2f}%'

            self.results_all = np.concatenate(results_all)
            self.gts_all = np.concatenate(gts_all)
            self.inputs_all = np.concatenate(inputs_all)
            
            results_dict = {'results_all': self.results_all, 'gts_all': self.gts_all, 'inputs_all': self.inputs_all}
            savepkl(results_dict, save_path)
            with self.result_out:
                print('result saved!')
            
        with self.result_out:
            print(f'Inference Done for {args.model}!')

        # get clip info
        _, split_id_test = self.datareader.get_split_id() # [range(0, 243) ... range(102759, 103002)] 
        actions = np.array(self.datareader.dt_dataset['test']['action']) # 103130 ['squat' ...  'kneeup']
        #factors = np.array(self.atareader.dt_dataset['test']['2.5d_factor']) # 103130 [3.49990559 ... 2.09230852]
        #gts = np.array(self.datareader.dt_dataset['test']['joints_2.5d_image']) # 103130, 17, 3
        sources = np.array(self.datareader.dt_dataset['test']['source']) # 103130 ['S02_6_squat_001' ... 'S08_4_kneeup_001']

        num_test_frames = len(actions)
        frames = np.array(range(num_test_frames))
        action_clips = np.array([actions[split_id_test[i]] for i in range(len(split_id_test))]) # actions[split_id_test]
        #factor_clips = np.array([factors[split_id_test[i]] for i in range(len(split_id_test))]) # factors[split_id_test]
        source_clips = np.array([sources[split_id_test[i]] for i in range(len(split_id_test))]) # sources[split_id_test]
        frame_clips  = np.array([frames[split_id_test[i]] for i in range(len(split_id_test))]) # frames[split_id_test]
        
        # print(action_clips.shape) # (B, T)
        # print(factor_clips.shape) # (B, T)
        self.action_clips = action_clips
        #self.factor_clips = factor_clips
        self.source_clips = source_clips
        self.frame_clips = frame_clips

        self.data_dict = {}
        source_list = natsorted(list(set(source_clips[:, 0]))) # (240)
        for source in source_list:
            batch_list = np.where(source_clips[:, 0] == source)[0]
            first_idx = batch_list[0]
            if self.target_dataset == 'H36M-SH':
                subject = f"S{int(source.split('_')[1])}"
                action = self.action_clips[first_idx, 0]
                cam = int(source.split('_')[-1])
            elif self.target_dataset == 'FIT3D'
                # test_s03_50591643_burpees
                subject = source.split('_')[1]
                action  = source.split('_')[3]
                cam = source.split('_')[2]

            if subject not in self.data_dict:
                self.data_dict[subject] = {}
            if action not in self.data_dict[subject]:
                self.data_dict[subject][action] = {}
            if cam not in self.data_dict[subject][action]:
                self.data_dict[subject][action][cam] = {}
            self.data_dict[subject][action][cam]['batch_list'] = batch_list
            self.data_dict[subject][action][cam]['inputs_all'] = self.inputs_all[batch_list]
            self.data_dict[subject][action][cam]['gts_all'] = self.gts_all[batch_list]
            self.data_dict[subject][action][cam]['results_all'] = self.results_all[batch_list]
        
        self.select_subject.options = list(self.data_dict.keys())
        if self.select_subject.value == '':
            self.select_subject.value = self.select_subject.options[0]
        self.select_action.options = list(self.data_dict[self.select_subject.value].keys())
        if self.select_action.value == '':
            self.select_action.value = self.select_action.options[0]
        self.select_cam.options = list(self.data_dict[self.select_subject.value][self.select_action.value].keys())
        if self.select_cam.value == '':
            self.select_cam.value = self.select_cam.options[0]
        self.select_batch.options = self.data_dict[self.select_subject.value][self.select_action.value][self.select_cam.value]['batch_list']
        if self.select_cam.value == '':
            self.select_batch.value = self.select_batch.options[0]

        # update
        self.results_all_shape.value = str(self.results_all.shape)
        self.gts_all_shape.value = str(self.gts_all.shape)
        self.inputs_all_shape.value = str(self.inputs_all.shape)
        self.frame_num.max = self.results_all.shape[1]
        self.play_vis.max = self.results_all.shape[1]
        self.update_batch_data(self.select_batch.value, self.button_root_rel.value, self.button_procrustes.value)

    def change_view(self, azim, elev, zoom):
        self.ax_3d1.azim = azim
        self.ax_3d1.elev = elev
        self.ax_3d1.set_xlim(-1/zoom, 1/zoom)
        self.ax_3d1.set_ylim(-1/zoom, 1/zoom)
        self.ax_3d1.set_zlim(-1/zoom, 1/zoom)

        self.ax_3d2.azim = azim + 90
        self.ax_3d2.elev = elev
        self.ax_3d2.set_xlim(-1/zoom, 1/zoom)
        self.ax_3d2.set_ylim(-1/zoom, 1/zoom)
        self.ax_3d2.set_zlim(-1/zoom, 1/zoom)

    def get_angle(self, batch_gt, batch_pred):
        self.angle_dict = {'results_angle': {}, 'gts_angle': {}}
        results_angle = get_limb_angle(batch_pred) # [8, F, 2]
        gts_angle = get_limb_angle(batch_gt) # [8, F, 2]
        self.angle_dict['results_angle']['R_UPPER_LEG'] = results_angle[0::8] # [1, F, 2]
        self.angle_dict['results_angle']['L_UPPER_LEG'] = results_angle[1::8]
        self.angle_dict['results_angle']['R_UPPER_ARM'] = results_angle[2::8]
        self.angle_dict['results_angle']['L_UPPER_ARM'] = results_angle[3::8]
        self.angle_dict['results_angle']['R_UNDER_LEG'] = results_angle[4::8]
        self.angle_dict['results_angle']['L_UNDER_LEG'] = results_angle[5::8]
        self.angle_dict['results_angle']['R_UNDER_ARM'] = results_angle[6::8]
        self.angle_dict['results_angle']['L_UNDER_ARM'] = results_angle[7::8]
        self.angle_dict['gts_angle']['R_UPPER_LEG'] = gts_angle[0::8]
        self.angle_dict['gts_angle']['L_UPPER_LEG'] = gts_angle[1::8]
        self.angle_dict['gts_angle']['R_UPPER_ARM'] = gts_angle[2::8]
        self.angle_dict['gts_angle']['L_UPPER_ARM'] = gts_angle[3::8]
        self.angle_dict['gts_angle']['R_UNDER_LEG'] = gts_angle[4::8]
        self.angle_dict['gts_angle']['L_UNDER_LEG'] = gts_angle[5::8]
        self.angle_dict['gts_angle']['R_UNDER_ARM'] = gts_angle[6::8]
        self.angle_dict['gts_angle']['L_UNDER_ARM'] = gts_angle[7::8]

    def update_select(self, subject, action, cam):
        try:
            if len(self.data_dict.keys()) > 0:
                with self.test_out:
                    print('update select')
                self.select_action.options = list(self.data_dict[subject].keys())
                self.select_action.value = action
                self.select_cam.options = list(self.data_dict[self.select_subject.value][action].keys())
                self.select_cam.value = cam
                self.select_batch.options = self.data_dict[self.select_subject.value][self.select_action.value][cam]['batch_list']
                self.select_batch.value = self.select_batch.options[0]
                # update batch data
                self.update_batch_data(self.select_batch.value, self.button_root_rel.value, self.button_procrustes.value)
        except Exception as e:
            with self.test_out:
                print('select error', e)
                print()

    def update_batch_data(self, batch, root_rel, procrustes):
        batch_num = self.select_batch.value
        try: 
            self.batch_input = self.inputs_all[batch_num].copy()
            batch_gt = self.gts_all[batch_num].copy()
            batch_pred = self.results_all[batch_num].copy()
            if root_rel:
                batch_gt = batch_gt - batch_gt[:, 0].reshape(-1, 1, 3)
                batch_pred = batch_pred - batch_pred[:, 0].reshape(-1, 1, 3)
            if procrustes:
                batch_pred = procrustes_align(batch_pred, batch_gt)
            self.batch_gt = batch_gt
            self.batch_pred = batch_pred
            # get limb length
            self.batch_gt_length = get_h36m_limb_lens(batch_gt.copy()) 
            self.batch_pred_length = get_h36m_limb_lens(batch_pred.copy())
            # get limb angle 
            self.get_angle(batch_gt, batch_pred)
            # calculate error
            error = np.linalg.norm(batch_gt - batch_pred, axis=-1) # [243, 17]
            self.error = error
            self.frame_level_error = error.mean(axis=-1) # [243]
            self.frame_level_var = error.std(axis=-1) # [243]# reinference for the changed model
            self.joint_level_error = error.mean(axis=0)
            self.joint_level_var = error.std(axis=0)
            self.batch_length_error = abs(self.batch_gt_length - self.batch_pred_length)        
            self.error_max_frame.value = str(self.frame_level_error.argmax())
            # update frame data
            self.prev_batch_analysis_dh = -1
            self.prev_frame_analysis_dh = -1
            self.prev_batch_analysis_error = -1
            self.prev_frame_analysis_error = -1
            self.update_frame_data(self.frame_slider.value)
        except Exception as e:
            print(e)

    def update_frame_data(self, frame): # self.frame_num.value
        try: 
            with self.test_out:
                print(frame, self.frame_slider.value, end='\r')
            frame_num = frame-1
            #self.frame_input_2d = self.batch_input[frame_num].copy()
            self.frame_gt = self.batch_gt[frame_num].copy()
            self.frame_pred = self.batch_pred[frame_num].copy()
            # rotate pose for better visualization
            if 'NO_FACTOR' not in self.dataset_list.value:
                self.frame_gt = rotate_torso_by_R(self.frame_gt, Rotation.from_rotvec([-np.pi/2, 0, 0]).as_matrix())
                self.frame_pred = rotate_torso_by_R(self.frame_pred, Rotation.from_rotvec([-np.pi/2, 0, 0]).as_matrix())
            self.joint_error_at_frame = self.error[frame_num].copy()
            # updtate figure
            self.visualize_2d_3d()
            self.update_analysis(self.select_part.value, self.button_analysis_error.value, self.button_analysis_dh.value)   
        except Exception as e:
            with self.test_out:
                print(e)


    def update_analysis(self, part_list, analyze_error, analyze_dh):
        try:
            frame_num = self.frame_slider.value - 1
            batch_num = self.select_batch.value
            part_ids = [len_ids[part] for part in part_list]
            length_error = abs(self.batch_gt_length - self.batch_pred_length)

            # plot error -----------------------------------------------------------------
            if analyze_error:
                if self.prev_batch_analysis_error != batch_num: batch_update = True
                else: batch_update = False
                if self.prev_frame_analysis_error != frame_num: frame_update = True
                else: frame_update = False
                with self.analysis_error:
                    if batch_update:
                        clear_axes([self.ax_error_per_frame, self.ax_error_per_joint])
                        self.ax_error_per_frame.plot(self.frame_level_error, 'r')
                        self.ax_error_per_frame.set_ylim(0, 0.1)# frame_level_error.max()*1.2)
                        self.ax_error_per_joint.bar(np.arange(len(self.joint_level_error)), self.joint_level_error, yerr=self.joint_level_var, color='r')
                        self.ax_error_per_joint.set_xticks(np.arange(len(self.joint_error_at_frame)))
                        self.ax_error_per_joint.set_xticklabels(self.joint_names, rotation=45)
                        self.ax_error_per_joint.set_ylim(0, self.error.max()*1.2)
                        self.analysis_error_per_joint_bar =  self.ax_error_per_joint.bar(np.arange(len(self.joint_error_at_frame)), self.joint_error_at_frame, width=0.5, alpha=0.5, color='b')
                        self.analysis_error_vline = self.ax_error_per_frame.vlines(self.frame_num.value, 0, self.frame_level_error.max()*1.2, colors='b', linestyles='dashed')
                    if frame_update:
                        self.analysis_error_vline.remove()
                        self.analysis_error_vline = self.ax_error_per_frame.vlines(self.frame_num.value, 0, self.frame_level_error.max()*1.2, colors='b', linestyles='dashed')
                        self.analysis_error_per_joint_bar.remove()
                        self.analysis_error_per_joint_bar =  self.ax_error_per_joint.bar(np.arange(len(self.joint_error_at_frame)), self.joint_error_at_frame, width=0.5, alpha=0.5, color='b')
                    self.prev_batch_analysis_error = batch_num
                    self.prev_frame_analysis_error = frame_num


            # plot dh -----------------------------------------------------------------
            if analyze_dh:
                if self.prev_batch_analysis_dh != batch_num: batch_update = True
                else: batch_update = False
                if len(part_ids) != len(self.prev_part_ids): batch_update = True
                if len(part_ids) == len(self.prev_part_ids):
                    for i, id in enumerate(part_ids):
                        if id != self.prev_part_ids[i]: batch_update = True
                if self.prev_frame_analysis_dh != frame_num: frame_update = True
                else: frame_update = False
                with self.analysis_dh:
                    if batch_update:
                        clear_axes([self.ax_length, self.ax_azim, self.ax_elev])
                        self.ax_length.hlines(0, 0, len(self.batch_gt_length), colors='k', linestyles='dashed')
                        self.length_vline = self.ax_length.vlines(frame_num, 0, self.batch_gt_length.max(), colors='b', linestyles='dashed')
                        self.azim_vline = self.ax_azim.vlines(frame_num, 0, np.pi, colors='b', linestyles='dashed')
                        self.elev_vline = self.ax_elev.vlines(frame_num, 0, np.pi, colors='b', linestyles='dashed')
                        for i, id in enumerate(part_ids):
                            part = part_list[i]
                            self.ax_length.plot(self.batch_gt_length[:, id], 'k--', label='gt')
                            self.ax_length.plot(self.batch_pred_length[:, id], 'y', label='pred')
                            self.ax_length.plot(length_error[:, id], label='abs error')
                            self.ax_length.set_title(f'{part} length error')

                            self.ax_azim.plot(self.angle_dict['gts_angle'][part][0, :, 0], 'k--', label='gt azim')
                            self.ax_azim.plot(self.angle_dict['results_angle'][part][0, :, 0], 'b', label='pred azim')
                            self.ax_azim.plot(self.angle_dict['gts_angle'][part][0, :, 0] - self.angle_dict['results_angle'][part][0, :, 0], label='error azim')
                            self.ax_azim.hlines(0, 0, len(self.angle_dict['gts_angle'][part][0]), colors='k', linestyles='dashed')
                            self.ax_azim.set_title(f'{part} azim angle')

                            self.ax_elev.plot(self.angle_dict['gts_angle'][part][0, :, 1], 'k--', label='gt elev')
                            self.ax_elev.plot(self.angle_dict['results_angle'][part][0, :, 1], 'r', label='pred elev')
                            self.ax_elev.plot(self.angle_dict['gts_angle'][part][0, :, 1] - self.angle_dict['results_angle'][part][0, :, 1], label='error elev')
                            self.ax_elev.hlines(0, 0, len(self.angle_dict['gts_angle'][part][0]), colors='k', linestyles='dashed')
                            self.ax_elev.set_title(f'{part} elev angle')   
                    if frame_update:
                        self.length_vline.remove()
                        self.length_vline = self.ax_length.vlines(frame_num, 0, self.batch_gt_length.max(), colors='b', linestyles='dashed')
                        self.azim_vline.remove()
                        self.azim_vline = self.ax_azim.vlines(frame_num, 0, np.pi, colors='b', linestyles='dashed')
                        self.elev_vline.remove()
                        self.elev_vline = self.ax_elev.vlines(frame_num, 0, np.pi, colors='b', linestyles='dashed')
                    self.prev_batch_analysis_dh = batch_num
                    self.prev_frame_analysis_dh = frame_num
                    self.prev_part_ids = part_ids
                
        except Exception as e:
            with self.test_out:
                print(e)

    def visualize_2d_3d(self):
        batch_num = self.select_batch.value
        frame_num = self.frame_slider.value - 1
        if type(self.batch_input) != type(None) and type(self.action_clips) != type(None) and type(self.frame_gt) != type(None) and type(self.frame_pred) != type(None):
            try: 
                clear_axes([self.ax_input, self.ax_3d1, self.ax_3d2])
                
                with self.plot_input:
                    if type(self.batch_input) != type(None):
                        draw_2d_pose(self.ax_input, self.batch_input[frame_num], normalize=True)
                with self.plot_3d:
                    if type(self.action_clips) != type(None):
                        self.ax_3d1.set_title(f'batch {batch_num} frame {frame_num} GT - action: {self.action_clips[batch_num, self.frame_num.value-1]}')
                    draw_3d_pose(self.ax_3d1, self.frame_gt, alpha=0.5, color='0')
                    draw_3d_pose(self.ax_3d1, self.frame_pred)
                    draw_3d_pose(self.ax_3d2, self.frame_gt, alpha=0.5, color='0')
                    draw_3d_pose(self.ax_3d2, self.frame_pred)
            except Exception as e:
                with self.test_out:
                    print(e)

    def clear_figure(self, b):
        with self.plot_input:
            plt.clf()
            plt.cla()
            self.ax_input.clear()
        with self.plot_3d:
            plt.clf()
            plt.cla()
            self.ax_3d1.clear()
            self.ax_3d2.clear()
        with self.analysis_error:
            plt.clf()
            plt.cla()
            self.ax_error_per_frame.clear()
            self.ax_error_per_joint.clear()
        with self.analysis_dh:
            plt.clf()
            plt.cla()
            self.ax_length.clear()
            self.ax_azim.clear()
            self.ax_elev.clear()
        self.plot_input.clear_output()
        self.plot_3d.clear_output()
        self.analysis_error.clear_output()
        self.analysis_dh.clear_output()
        self.vis_plot_init()
        self.analysis_plot_init()        
        self.visualize_2d_3d()
        self.view_reset(0)
        self.prev_batch_analysis_dh = -1
        self.prev_frame_analysis_dh = -1
        self.prev_batch_analysis_error = -1
        self.prev_frame_analysis_error = -1
        self.update_analysis(self.select_part.value, self.button_analysis_error.value, self.button_analysis_dh.value)

    def interactive(self):
        self.load_model_interact = widgets.interactive_output(self.load_model, {'target_model': self.model_list})
        self.load_dataset_interact = widgets.interactive_output(self.load_data, {'target_dataset': self.dataset_list})
        self.button_inference.click()
        self.batch_interact = widgets.interactive(self.update_batch_data, batch=self.select_batch, root_rel=self.button_root_rel, procrustes=self.button_procrustes)
        self.select_interact = widgets.interactive(self.update_select, subject=self.select_subject, action=self.select_action, cam=self.select_cam)
        self.frame_interact = widgets.interactive(self.update_frame_data, frame=self.frame_slider)
        self.analysis_interact = widgets.interactive(self.update_analysis, part_list=self.select_part, analyze_error=self.button_analysis_error, analyze_dh=self.button_analysis_dh)
        self.view_interact = widgets.interactive(self.change_view, azim=self.azim, elev=self.elev, zoom=self.zoom)

    def layout(self):
        ui_prepare = widgets.VBox([self.model_list, self.dataset_list, self.load_model_interact, self.load_dataset_interact], layout=box_layout, width='auto')
        ui_inference = widgets.VBox([self.button_inference, self.inference_progress, self.result_out, self.inputs_all_shape, self.gts_all_shape, self.results_all_shape], 
                                    layout=box_layout, width='auto')
        ui_select = widgets.HBox([self.select_subject, self.select_action, self.select_cam, self.select_batch, self.select_part], layout=box_layout)
        # 
        ui_cam_control = widgets.VBox([self.azim, self.elev, self.zoom], width='auto')
        ui_vis_option = widgets.HBox([self.button_root_rel, self.button_procrustes, self.button_analysis_error, self.button_analysis_dh], width='auto')
        ui_reset_button = widgets.HBox([self.button_frame_reset, self.button_view_reset, self.button_delay_reset, self.button_figure_reset], width='auto')
        ui_play_control = widgets.VBox([self.play_vis, self.delay, self.error_max_frame, self.button_go_to_max_frame])
        ui_vis_control = widgets.VBox([self.frame_num, ui_cam_control, ui_vis_option, ui_reset_button, ui_play_control], layout=box_layout, width='auto')
        ui_figure = widgets.HBox([self.plot_input, widgets.VBox([self.plot_3d, self.frame_slider])], layout=box_layout, width='auto')
        # 
        self.frame_slider.layout.width = 'auto' #'1500px'
        ui_analysis_error = widgets.VBox([self.analysis_error], width='auto')
        ui_analysis_dh = widgets.VBox([self.analysis_dh], width='auto')
        #ui_analysis_control = widgets.VBox([], layout=box_layout, width='auto')
        ui_analysis = widgets.HBox([ui_analysis_error, ui_analysis_dh], layout=box_layout, width='auto')
        
        # #
        # self.button_test1 = widgets.Button(description='go to ui2')
        # self.button_test1.on_click(self.go_to_ui1)
        # self.button_test2 = widgets.Button(description='go to ui1')
        # self.button_test2.on_click(self.main)


        # total ui
        ui_level1 = widgets.HBox([ui_prepare, ui_inference, ui_select, ui_vis_control])
        ui_level2 = widgets.HBox([ui_figure, ui_analysis])        
        self.ui = widgets.VBox([ui_level1, ui_level2, self.test_out])
        #self.test_ui = widgets.HBox([ui_prepare, ui_inference, ui_select, ui_vis_control, self.test_out])
        #self.ui2 = widgets.Box([self.button_test2])
        #self.main_display = widgets.Output()

    def update_batch_data2(self, batch):
        try:
            with self.test_out:
                print('batch update')
        except Exception as e:
            with self.test_out:
                print('batch error', e)

    def test_output(self, subject, action, cam, batch, part_list):
        with self.test_out:
            #print(subject, action, cam, batch, part_list)
            print(batch)

    
    def main(self, b=0):
        self.button_root_rel.value = True
        self.button_analysis_dh.value = True
        self.button_analysis_error.value = True
        # self.main_display.clear_output()
        # with self.main_display:
        dis = display(self.ui)
        #display(self.main_display)

at = analysis_tool()
 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


s_09_act_02_subact_01_ca_01
s_09_act_02_subact_01_ca_02
s_09_act_02_subact_01_ca_03
s_09_act_02_subact_01_ca_04
s_09_act_02_subact_02_ca_01
s_09_act_02_subact_02_ca_02
s_09_act_02_subact_02_ca_03
s_09_act_02_subact_02_ca_04
s_09_act_03_subact_01_ca_01
s_09_act_03_subact_01_ca_02
s_09_act_03_subact_01_ca_03
s_09_act_03_subact_01_ca_04
s_09_act_03_subact_02_ca_01
s_09_act_03_subact_02_ca_02
s_09_act_03_subact_02_ca_03
s_09_act_03_subact_02_ca_04
s_09_act_04_subact_01_ca_01
s_09_act_04_subact_01_ca_02
s_09_act_04_subact_01_ca_03
s_09_act_04_subact_01_ca_04
s_09_act_04_subact_02_ca_01
s_09_act_04_subact_02_ca_02
s_09_act_04_subact_02_ca_03
s_09_act_04_subact_02_ca_04
s_09_act_05_subact_01_ca_01
s_09_act_05_subact_01_ca_02
s_09_act_05_subact_01_ca_03
s_09_act_05_subact_01_ca_04
s_09_act_05_subact_02_ca_01
s_09_act_05_subact_02_ca_02
s_09_act_05_subact_02_ca_03
s_09_act_05_subact_02_ca_04
s_09_act_06_subact_01_ca_01
s_09_act_06_subact_01_ca_02
s_09_act_06_subact_01_ca_03
s_09_act_06_subact_0

test_s03_50591643_burpees


ValueError: invalid literal for int() with base 10: 's03'